In [1]:
import logging
import warnings

logging.basicConfig()
logging.getLogger().setLevel(logging.INFO)

warnings.filterwarnings("ignore")

In [2]:
from freamon.adapters.mlinspect.provenance import from_py_file

view_gen = from_py_file('pipelines--mlinspect--credit.py', cmd_args=[])

INFO:root:Patching sys.argv with ['eyes']
INFO:root:Executing instrumented user pipeline with mlinspect
INFO:root:Redirecting the pipeline's stdout to pipeline-output.txt
INFO:root:---RUNTIME: Instrumented execution took 4049.7570037841797 ms
INFO:root:Registering test source 1 with columns: ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income-per-year', 'mlinspect_lineage']


In [4]:
df = view_gen.test_view(sliceable_by=['race', 'sex'], with_features=False, with_y_true=True, with_y_pred=True)

INFO:root:
SELECT fs1.race, fs1.sex, ftv.y_true, ftv.y_pred
FROM _freamon_test_view ftv
JOIN _freamon_source_1_with_prov_view fs1  ON fs1.prov_id_source_1 = ftv.prov_id_source_1
   
        


In [5]:
view_gen.db.execute(
"""
SELECT 
    (race!='Black' OR sex!='Male') AS privileged, 
    SUM(CAST((y_true=1 AND y_pred=1) AS INTEGER)) AS true_positive,
    SUM(CAST((y_true=1 AND y_pred=0) AS INTEGER)) AS false_negative,    
    SUM(CAST((y_true=0 AND y_pred=1) AS INTEGER)) AS false_positive,    
    SUM(CAST((y_true=0 AND y_pred=0) AS INTEGER)) AS true_negative,    
FROM df
GROUP BY privileged
"""
).df()

,privileged,true_positive,false_negative,false_positive,true_negative
0,True,192.0,235.0,126.0,1437.0
1,False,9.0,7.0,18.0,104.0
